In [1]:
import requests
from bs4 import BeautifulSoup
import os
import xlsxwriter
import xlrd

In [2]:
def get_link(keyword):
    keyword_link = []
    url = "https://law.moj.gov.tw/Law/LawSearchResult.aspx?ty=ONEBAR&kw="
    for i in keyword:
        keyword_link.append(url+i)
    return keyword_link

In [3]:
def get_html(url):
    #模擬訪問頁面的函數
    try:
        user_agent = 'Mozilla/5.0'
        resp = requests.get(url, headers={'User-Agent': user_agent}, timeout = 30) #回傳為一個request.Response的物件
        resp.endcoding = 'utf8'
        return resp.text
    except:
        return 'ERROR'

In [4]:
def get_law(url):
    try:
        law_name = {}
        label_fei = {}
        flag=1
        while flag==1:
            html = get_html(url)
            soup = BeautifulSoup(html,"html.parser")
            sel = soup.select("#pnLaw td")
            next_page = soup.select("ul.pager li #hlPage")
            if str(next_page[2]).find('aspNetDisabled')!=-1:
                flag = 0
            else:
                url = 'https://law.moj.gov.tw'+next_page[2]["href"]
            for i in range(len(sel)):
                if sel[i].text[0]!='\r':
                    law = sel[i].text.strip('\n')
                    law = law.replace('\r' and '\n',' ')
                    a = law.split('\r')
                    a[1] = a[1].strip(' ')
                    a[1] = a[1].strip('英')
                    if a[0][0]=='廢':
                        label_fei[a[0].strip('廢')] = a[1]
                    else:
                        law_name[a[0]] = a[1]
        return [law_name,label_fei]
    except:
        return '您所輸入的查詢條件查無資料，請重新設定查詢條件！'

In [5]:
def get_all_law(keyword):
    Dict_keyword = {}
    keyword_link = get_link(keyword)
    for i in range(len(keyword_link)):
        law = get_law(keyword_link[i])
        Dict_keyword[keyword[i]] = law
    return Dict_keyword

In [6]:
def write_in_excel(keyword,Dict_keyword):
    workbook = xlsxwriter.Workbook('法條整理.xlsx')
    worksheet = []
    first_ws = workbook.add_worksheet('目錄')
    row=0
    for i in range(len(keyword)):
        first_ws.write(row,0,keyword[i])
        row+=1
        worksheet.append(workbook.add_worksheet(keyword[i]))
    for i,key in enumerate(keyword):
        worksheet[i].write(0,0,key+'相關法條')
        if type(Dict_keyword[key])==str:
            worksheet[i].write(1,0,'您所輸入的查詢條件查無資料，請重新設定查詢條件！')
        else:
            row=1
            for j in range(len(Dict_keyword[key][0])):
                worksheet[i].write(row,0,list(Dict_keyword[key][0].keys())[j])
                worksheet[i].write(row,1,list(Dict_keyword[key][0].values())[j])
                row+=1
            row+=1
            worksheet[i].write(row,0,"已廢除：")
            row+=1
            for j in range(len(Dict_keyword[key][1])):
                worksheet[i].write(row,0,list(Dict_keyword[key][1].keys())[j])
                worksheet[i].write(row,1,list(Dict_keyword[key][1].values())[j])
                row+=1
    workbook.close()

In [7]:
def read_from_excel():
    ad_wb = xlrd.open_workbook("法條整理.xlsx")
    entry = ad_wb.sheet_by_name('目錄')
    row=0
    keyword = entry.col_values(0)
    return keyword

In [8]:
#main
keyword = read_from_excel()
Dict_keyword = get_all_law(keyword)
write_in_excel(keyword,Dict_keyword)